Test 1 - (a)

In [ ]:
import subprocess
import json
from datetime import datetime

In [2]:
# Function to get service status
def get_service_status(service_name):
    status = "DOWN"
    try:
        # Run systemctl command to check service status
        output = subprocess.check_output(["systemctl", "is-active", service_name]).decode().strip()
        if output == "active":
            status = "UP"
    except subprocess.CalledProcessError:
        pass
    return status

In [3]:
# Function to create JSON object
def create_json(service_name, service_status, host_name):
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    json_data = {
        "service_name": service_name,
        "service_status": service_status,
        "host_name": host_name
    }
    file_name = f"{service_name}-status-{timestamp}.json"
    with open(file_name, 'w') as file:
        json.dump(json_data, file, indent=4)
    print(f"JSON object created and saved to {file_name}")

In [8]:
# Define the service names
services = ["httpd", "rabbitMQ", "postgreSQL"]
host_name = "host1"  

In [ ]:
# Iterate through services
for service in services:
    status = get_service_status(service)
    create_json(service, status, host_name)

Test1 - (b)

In [ ]:
from flask import Flask, request, jsonify
from elasticsearch import Elasticsearch

In [ ]:
app = Flask(__name__)

In [ ]:
# Connect to Elasticsearch
es = Elasticsearch('localhost:9200')

In [ ]:
# Endpoint to insert payload into Elasticsearch
@app.route('/add', methods=['POST'])
def add_to_elasticsearch():
    try:
        data = request.get_json()
        # Insert payload into Elasticsearch
        es.index(index='service_status', body=data)
        return jsonify({"message": "Data added to Elasticsearch"}), 200
    except Exception as e:
        return jsonify({"message": str(e)}), 500

In [ ]:
# Endpoint to get overall application status (UP or DOWN)
@app.route('/healthcheck', methods=['GET'])
def overall_healthcheck():
    try:
        # Elasticsearch query to get overall application status
        res = es.search(index='service_status', body={"query": {"match_all": {}}})
        hits = res['hits']['hits']
        status = {"UP": 0, "DOWN": 0}
        for hit in hits:
            service_status = hit['_source']['service_status']
            status[service_status] += 1
        if status["DOWN"] > 0:
            overall_status = "DOWN"
        else:
            overall_status = "UP"
        return jsonify({"overall_status": overall_status}), 200
    except Exception as e:
        return jsonify({"message": str(e)}), 500

In [ ]:
# Endpoint to get specific application status (UP or DOWN) based on service name
@app.route('/healthcheck/<service_name>', methods=['GET'])
def specific_healthcheck(service_name):
    try:
        # Elasticsearch query to get specific application status
        res = es.search(index='service_status', body={"query": {"match": {"service_name": service_name}}})
        hits = res['hits']['hits']
        if hits:
            service_status = hits[0]['_source']['service_status']
            return jsonify({"service_name": service_name, "service_status": service_status}), 200
        else:
            return jsonify({"message": "Service not found"}), 404
    except Exception as e:
        return jsonify({"message": str(e)}), 500

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)  

Test 2 - (a)